# Histograms of Colors

In [17]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline,FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import make_column_transformer, make_column_selector
from custom import ColorHistogram, Splitter, CustomNormalizer, Loader
from sklearn import set_config
set_config(display="diagram")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Creation

We first import the metadata and load only the images relative to `dev set` due to shortages of computation power.

In [9]:
metadata = pd.read_csv('dataset/metadata.csv', index_col=['src','slice_num'])
# metadata.loc[metadata.partition=='dev','features'] = metadata[metadata.partition=='dev'].img_slice.apply(lambda x: np.load(x))
dev_data = metadata[metadata.partition=='dev']
dev_data.head()

img_slice  ... label
src slice_num                                             ...      
0   79         og_dataset/splits/dev/slice_0_img_079.npy  ...   0.0
    121        og_dataset/splits/dev/slice_0_img_121.npy  ...   0.0
    174        og_dataset/splits/dev/slice_0_img_174.npy  ...   0.0
2   10         og_dataset/splits/dev/slice_2_img_010.npy  ...   0.0
    16         og_dataset/splits/dev/slice_2_img_016.npy  ...   0.0

[5 rows x 10 columns]

### Pipeline
Now we create the pipeline that will handle our data and process each image into a set of features.

In [ ]:
feature_names = ['LE7 B1 (blue)', 'LE7 B2 (green)', 'LE7 B3 (red)', 'LE7 B4 (near infrared)', 'LE7 B5 (shortwave infrared 1)', 'LE7 B6_VCID_1 (low-gain thermal infrared)', 'LE7 B6_VCID_2 (high-gain thermal infrared)', 'LE7 B7 (shortwave infrared 2)', 'LE7 B8 (panchromatic)', 'LE7 BQA (quality bitmask)', 'NDVI (vegetation index)', 'NDSI (snow index)', 'NDWI (water index)', 'SRTM 90 elevation', 'SRTM 90 slope']

normalization_file = 'og_dataset/stats_train.json'
import json
with open(normalization_file) as f:
    channels_stats = json.load(f)

pipe = make_column_transformer(
    (make_pipeline(
        CustomNormalizer(channels_stats['means'],channels_stats['stds']),
        Loader(),
        Splitter(feature_names),
        ColorHistogram()
    ), ['img_slice']),
)

histograms = pipe.fit_transform(dev_data)
histograms

<class 'pandas.core.frame.DataFrame'>


For each of the layers we can visualize the bin edges:

In [ ]:
pipe.named_transformers_['pipeline'].named_steps['colorhistogram'].bins

And also the number of bins for each layer:

In [31]:
print('Bin size for each column')
fitted_bins = pipe.transformers_[0][1].steps[1][1].bins
for col, bin in fitted_bins.items():
    print(f'{col:45} {len(bin)}')

Bin size for each column
LE7 B1 (blue)                                 39
LE7 B2 (green)                                39
LE7 B3 (red)                                  40
LE7 B4 (near infrared)                        39
LE7 B5 (shortwave infrared 1)                 39
LE7 B6_VCID_1 (low-gain thermal infrared)     37
LE7 B6_VCID_2 (high-gain thermal infrared)    38
LE7 B7 (shortwave infrared 2)                 39
LE7 B8 (panchromatic)                         39
LE7 BQA (quality bitmask)                     38
NDVI (vegetation index)                       37
NDSI (snow index)                             38
NDWI (water index)                            37
SRTM 90 elevation                             39
SRTM 90 slope                                 38


Finally we assemble everything into a nice and tidy pandas Dataframe.

In [32]:
bin_lens = {col:len(bin)-1 for col, bin in fitted_bins.items()}
new_cols = [(col, i) for col, length in bin_lens.items() for i in range(length)]
d_hist = pd.DataFrame(histograms, columns=pd.MultiIndex.from_tuples(new_cols))
d_hist.index = dev_data.index
d_hist.head()

LE7 B1 (blue)                                                    \
                         0         1         2         3         4         5    
src slice_num                                                                   
0   79         0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000   
    121        5.603266e-07  0.000007  0.000009  0.000006  0.000007  0.000008   
    174        0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000   
2   10                  NaN       NaN       NaN       NaN       NaN       NaN   
    16                  NaN       NaN       NaN       NaN       NaN       NaN   

                                                       ... SRTM 90 slope       \
                     6         7         8         9   ...            27   28   
src slice_num                                          ...                      
0   79         0.000000  0.000000  0.000004  0.009719  ...      0.000000  0.0   
    121        0.000007  0.000008  0.000010  0.000015  ...      0.000000  0.0   
    174        0.000000  0.000000  0.000000  0.000000  ...      0.000000  0.0   
2   10              NaN       NaN       NaN       NaN  ...      0.000000  0.0   
    16              NaN       NaN       NaN       NaN  ...      0.000019  0.0   

                                                       
                29   30   31   32   33   34   35   36  
src slice_num                                          
0   79         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
    121        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
    174        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2   10         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
    16         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 561 columns]

Then we save everything to disk. To read it:
```python
pd.read_csv('dataset/hoc.csv', header=[0,1], index_col=[0,1])
```

In [45]:
d_hist.to_csv('dataset/hoc.csv')

## Analysis

There's a good chunk of NaN values in the final result.

In [33]:
print('Percentage of NaNs: %s%%' % round(d_hist.isna().sum().sum() / d_hist.shape[0] / d_hist.shape[1] *100, 2))

Percentage of NaNs: 14.78%


In [37]:
for name in feature_names:
    nan_val = d_hist.loc[:, name].isna().sum().iloc[0]
    print(name.ljust(45), nan_val)

LE7 B1 (blue)                                 18
LE7 B2 (green)                                19
LE7 B3 (red)                                  19
LE7 B4 (near infrared)                        19
LE7 B5 (shortwave infrared 1)                 18
LE7 B6_VCID_1 (low-gain thermal infrared)     19
LE7 B6_VCID_2 (high-gain thermal infrared)    19
LE7 B7 (shortwave infrared 2)                 18
LE7 B8 (panchromatic)                         18
LE7 BQA (quality bitmask)                     20
NDVI (vegetation index)                       19
NDSI (snow index)                             19
NDWI (water index)                            19
SRTM 90 elevation                             0
SRTM 90 slope                                 0


In [46]:
dev_data.head(10)

img_slice  \
src slice_num                                              
0   79         og_dataset/splits/dev/slice_0_img_079.npy   
    121        og_dataset/splits/dev/slice_0_img_121.npy   
    174        og_dataset/splits/dev/slice_0_img_174.npy   
2   10         og_dataset/splits/dev/slice_2_img_010.npy   
    16         og_dataset/splits/dev/slice_2_img_016.npy   
    47         og_dataset/splits/dev/slice_2_img_047.npy   
    173        og_dataset/splits/dev/slice_2_img_173.npy   
    207        og_dataset/splits/dev/slice_2_img_207.npy   
3   56         og_dataset/splits/dev/slice_3_img_056.npy   
    65         og_dataset/splits/dev/slice_3_img_065.npy   

                                               mask_slice  glaciers  \
src slice_num                                                         
0   79         og_dataset/splits/dev/slice_0_mask_079.npy  0.000000   
    121        og_dataset/splits/dev/slice_0_mask_121.npy  0.000000   
    174        og_dataset/splits/dev/slice_0_mask_174.npy  0.000000   
2   10         og_dataset/splits/dev/slice_2_mask_010.npy  0.000000   
    16         og_dataset/splits/dev/slice_2_mask_016.npy  0.000000   
    47         og_dataset/splits/dev/slice_2_mask_047.npy  0.246052   
    173        og_dataset/splits/dev/slice_2_mask_173.npy  0.191116   
    207        og_dataset/splits/dev/slice_2_mask_207.npy  0.116890   
3   56         og_dataset/splits/dev/slice_3_mask_056.npy  0.376938   
    65         og_dataset/splits/dev/slice_3_mask_065.npy  0.000000   

               clean_glaciers  debris_glaciers    img_mean            lng  \
src slice_num                                                               
0   79               0.000000         0.000000  142.014130  394523.684211   
    121              0.000000         0.000000  113.703094  348977.894737   
    174              0.000000         0.000000  112.221992  470433.333333   
2   10               0.000000         0.000000  333.149292  317285.107228   
    16               0.000000         0.000000  317.823853  180647.872461   
    47               0.225334         0.020718  493.879822  195829.787435   
    173              0.173862         0.017254  515.300476  286921.277280   
    207              0.105595         0.011295  450.122375  347648.937177   
3   56               0.347935         0.029003  495.190918  760657.583441   
    65               0.000000         0.000000  253.115891  669565.800776   

                        lat partition  label  \
src slice_num                                  
0   79         3.648087e+06       dev    0.0   
    121        3.693633e+06       dev    0.0   
    174        3.739179e+06       dev    0.0   
2   10         3.414856e+06       dev    0.0   
    16         3.430038e+06       dev    0.0   
    47         3.460402e+06       dev    1.0   
    173        3.581859e+06       dev    1.0   
    207        3.612224e+06       dev    1.0   
3   56         3.462743e+06       dev    1.0   
    65         3.477925e+06       dev    0.0   

                                                        features  
src slice_num                                                     
0   79         [[[67.0, 49.0, 41.0, 45.0, 46.0, 130.0, 148.0,...  
    121        [[[88.0, nan, nan, nan, nan, nan, nan, nan, 58...  
    174        [[[73.0, 62.0, 50.0, 80.0, 73.0, 136.0, 159.0,...  
2   10         [[[nan, nan, nan, nan, nan, nan, nan, nan, nan...  
    16         [[[nan, nan, nan, nan, nan, nan, nan, nan, nan...  
    47         [[[-1.7699648, -1.5762353, -1.6766024, -1.5867...  
    173        [[[-0.023158196, -0.29675427, -0.3828696, -0.7...  
    207        [[[1.1996064, 1.2608747, 1.2172735, 1.615497, ...  
3   56         [[[1.1996064, 1.2608747, 1.2172735, 0.42251736...  
    65         [[[50.0, 39.0, 34.0, 57.0, 65.0, 121.0, 132.0,...